In [ ]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import time

apikey = "rG3tQa9lws5UNKCLP1poUjP65xVSe_Wl"
alpha_key = "TGVHMDFHS5MWFDXF"
sec_key = 'e0a0986222f34fbdcd03fd20258d687bcc2da4fa3311ca77595a855750728bea'

In [ ]:
polygon = requests.get("https://api.polygon.io/v2/aggs/ticker/SPY/range/1/day/2000-07-01/2022-03-01?limit=5000&apiKey=%s" % (apikey))

In [ ]:
inflation = requests.get('https://www.alphavantage.co/query?function=INFLATION_EXPECTATION&apikey=%s' % alpha_key)
treasury_yields = requests.get("https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&apikey=%s" % alpha_key)
unemployment = requests.get("https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey=%s" % alpha_key)

In [ ]:
locs = requests.get("https://api.sec-api.io/mapping/exchange/BATS?token=%s" % sec_key)

In [ ]:
daily_data = pd.json_normalize(polygon.json(), ["results"])
daily_data["t"] = pd.to_datetime(daily_data["t"].apply(lambda x: datetime.fromtimestamp(x / 1000).strftime("%Y-%m-%d")))

In [ ]:
inflation_data = pd.json_normalize(inflation.json(), ["data"])
yields_data = pd.json_normalize(treasury_yields.json(), ["data"])
employment_data = pd.json_normalize(unemployment.json(), ["data"])
location_data = pd.json_normalize(locs.json())
inflation_data["date"] = pd.to_datetime(inflation_data["date"])
yields_data["date"] = pd.to_datetime(yields_data["date"])
employment_data["date"] = pd.to_datetime(employment_data["date"])

In [ ]:
dates_vals_inf = []
dates_vals_emp = []

for i in range(len(inflation_data[inflation_data["date"] >= datetime(2020, 3, 1)]) - 1):
  time_range = pd.date_range(inflation_data.loc[i+1].date, inflation_data.loc[i].date)
  dates_vals_inf.append(pd.Series(len(time_range) * [inflation_data.loc[i].value], index=time_range))

for i in range(len(employment_data[employment_data["date"] >= datetime(2020, 3, 1)]) - 1):
  time_range = pd.date_range(employment_data.loc[i+1].date, employment_data.loc[i].date)
  dates_vals_emp.append(pd.Series(len(time_range) * [employment_data.loc[i].value], index=time_range))

In [ ]:
inflation_series = dates_vals_inf[0]
employment_series = dates_vals_emp[0]

for series in dates_vals_inf[1:]:
  inflation_series = pd.concat([inflation_series, series])

for series in dates_vals_emp[1:]:
  employment_series = pd.concat([employment_series, series])

inf_df = inflation_series.sort_index(ascending=True).to_frame(name="monthly_inflation_exp")
emp_df = employment_series.sort_index(ascending=True).to_frame(name="employment_rate")
inf_df = inf_df.reset_index().rename(columns={"index": "date"})
emp_df = emp_df.reset_index().rename(columns={"index": "date"})

In [ ]:
all = daily_data.copy()
all["t"] = pd.to_datetime(all["t"])
all = all.merge(yields_data[yields_data["date"] >= datetime(2020, 3, 1)], how="left", left_on="t", right_on="date").drop("date", axis=1).rename(columns={"value": "10Y yield"})
all = all.merge(inf_df, how="left", left_on="t", right_on="date").drop("date", axis=1)
stock_df = all.merge(emp_df, how="left", left_on="t", right_on='date').drop("date", axis=1)
stock_df["monthly_inflation_exp"] = stock_df["monthly_inflation_exp"].replace(np.nan, 5)
stock_df.employment_rate = stock_df["employment_rate"].replace(np.nan, 3.6).astype(float)

In [ ]:
stock_df["daily return"] = stock_df["c"].pct_change()
stock_df["daily return"] = stock_df["daily return"].replace(np.nan, 0)
stock_df["10Y yield"] = stock_df["10Y yield"].replace(".", np.nan)
stock_df = stock_df.dropna()
dates = stock_df.set_index("t").index
stock_df["time"] = dates.year + (30 * (dates.month - 1) + dates.day) / 365

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
stock_df.to_csv("stock_df.csv")

In [ ]:
companies = pd.json_normalize(locs.json())
prices = []
locations = companies[companies["location"] != ""]

for tick in locations["ticker"][:200]:
  respose = requests.get("https://api.polygon.io/v2/aggs/ticker/%s/prev?limit=5000&apiKey=%s" % (tick, apikey))
  try:
    price = pd.json_normalize(respose.json(), ["results"])["c"][0]
    prices.append(price)
  except KeyError:
    prices.append(np.nan)
  time.sleep(12)


In [ ]:
locations = locations.iloc[:200]
locations["price"] = prices
locations.to_csv("locations.csv")

In [ ]:
ticker_lst = ["AAPL", "MSFT", "NVDA", "HD", "COST", "T", "TSLA", "F", "KO", "NKE", "JPM", "XOM", "GS", "JNJ", "CNC", "ABBV", "DOW", "GE", "BBY", "TWTR", "D", "VZ", "AMAT", "BA", "DAL", "PARA", "EXPE", "DIS", "BYND"]
metrics = []

df_metrics = pd.json_normalize(requests.get("https://www.alphavantage.co/query?function=OVERVIEW&symbol=AAPL&apikey=%s" % alpha_key).json())

for tick in ticker_lst[1:]:
  response = requests.get("https://www.alphavantage.co/query?function=OVERVIEW&symbol=%s&apikey=%s" % (tick, alpha_key))
  df = pd.json_normalize(response.json())
  df_metrics = pd.concat([df_metrics, df])
  time.sleep(12)

In [ ]:
df_metrics.to_csv("metrics.csv")

In [ ]:
tsla = pd.read_csv("TSLA_prices.csv", parse_dates=True)
att = pd.read_csv("T_prices.csv", parse_dates=True)
ko = pd.read_csv("KO_prices.csv", parse_dates=True)

In [ ]:
tsla["t"] = pd.to_datetime(tsla["Date"]) 
att["t"] = pd.to_datetime(att["Date"]) 
ko["t"] = pd.to_datetime(ko["Date"])

dates = tsla.set_index("t").index
tsla["time"] = dates.year + (30 * (dates.month - 1) + dates.day) / 365
dates = att.set_index("t").index
att["time"] = dates.year + (30 * (dates.month - 1) + dates.day) / 365
dates = ko.set_index("t").index
ko["time"] = dates.year + (30 * (dates.month - 1) + dates.day) / 365

In [ ]:
tsla["daily return"] = tsla["% Return"].replace(np.nan, 0)
att["daily return"] = att["% Return"].replace(np.nan, 0)
ko["daily return"] = ko["% Return"].replace(np.nan, 0)

In [ ]:
tsla["Price"] = tsla["Price"].replace(",", "")
att["Price"] = att["Price"].replace(",", "")
ko["Price"] = ko["Price"].replace(",", "")

In [ ]:
tsla.to_csv("tsla.csv")
att.to_csv("att.csv")
ko.to_csv("ko.csv")